In [1]:
%cd ../..

/Users/kochiallagbe/Documents/GitHub/PyTransit-MicroServices


In [2]:
import os
from pprint import pprint
from pymongo import MongoClient

from BIXI_Services.BIXI_Historical_Data_Checker.main import handler as checker_handler
from BIXI_Services.BIXI_Historical_Data_Processor.main import Config
from BIXI_Services.BIXI_Historical_Data_Processor.main import handler as processor_handler

from dotenv import load_dotenv


In [3]:
load_dotenv()
# os.environ["ATLAS_URI"] = 
os.environ["MONGO_DATABASE_NAME"] = "prod-monitoring-mtl"
os.environ["BIXI_URL_COLLECTION"] = "historic_data_urls"
os.environ["BIXI_LOCATION_COLLECTION"] = "station_locations"
os.environ["BIXI_TRIP_COLLECTION"] = "trips"
os.environ["BIXI_DATA_URL"] = "https://bixi.com/en/open-data"
os.environ["BIXI_CDN"] = "https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/"
os.environ["BIXI_DEFAULT_EXTRACT_PATH"] = "/tmp/data/"
os.environ["BIXI_CHUNK_SIZE"] = "1000000"


# Historical data initial db operations

## Initialize DB objects

‼️ `db.drop_collection(collection_name)` deletes all data.

In [4]:
config = Config(**os.environ)
client = MongoClient(config.ATLAS_URI)
db = client[config.MONGO_DATABASE_NAME]
for collection_name in db.list_collection_names():
    db.drop_collection(collection_name)

In [5]:
os.environ["BIXI_DEFAULT_EXTRACT_PATH"]
os.environ["MONGO_DATABASE_NAME"]

'prod-monitoring-mtl'

## Get the urls

In [6]:
checker_results = checker_handler(None, None)
urls = checker_results["urls"]
pprint(urls)

checking for new historic data..
scraped_urls {2014: 'https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/wp-content/uploads/2023/06/Historique-BIXI-2014.zip', 2015: 'https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/wp-content/uploads/2023/06/Historique-BIXI-2015.zip', 2016: 'https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/wp-content/uploads/2023/06/Historique-BIXI-2016.zip', 2017: 'https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/wp-content/uploads/2023/06/Historique-BIXI-2017.zip', 2018: 'https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/wp-content/uploads/2023/06/Historique-BIXI-2018.zip', 2019: 'https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/wp-content/uploads/2023/06/Historique-BIXI-2019.zip', 2020: 'https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/wp-content/uploads/2023/06/Historique-BIXI-2020.zip', 2021: 'https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/wp-content/uploads/2023/06/Historique-BIXI-2021.zip', 2022: 'https://s3.ca-central-1.amazonaws.com/cdn.bixi.com

## ETL

The format of the historic data have changed over the years. We've implemented an ETL strategy for the current format which is in effect since 2022. Urls for prior years will not be processed until we've created appropriate ETL strategies and configured them in `BIXI_Services.BIXI_Historical_Data_Processor.etl.transform_load.context` module.

In [7]:
if urls:
    processor_results = processor_handler({"urls": urls}, None)
    pprint(processor_results)

historic data processing started.
ETL process started for URL: https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/wp-content/uploads/2023/06/Historique-BIXI-2014.zip
start download and extract https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/wp-content/uploads/2023/06/Historique-BIXI-2014.zip
extracted files: ['/tmp/data/BixiMontrealRentals2014/OD_2014-04.csv', '/tmp/data/BixiMontrealRentals2014/OD_2014-05.csv', '/tmp/data/BixiMontrealRentals2014/OD_2014-06.csv', '/tmp/data/BixiMontrealRentals2014/OD_2014-07.csv', '/tmp/data/BixiMontrealRentals2014/OD_2014-08.csv', '/tmp/data/BixiMontrealRentals2014/OD_2014-09.csv', '/tmp/data/BixiMontrealRentals2014/OD_2014-10.csv', '/tmp/data/BixiMontrealRentals2014/OD_2014-11.csv', '/tmp/data/BixiMontrealRentals2014/Stations_2014.csv']
strategy not implemented for year 2014
ETL process started for URL: https://s3.ca-central-1.amazonaws.com/cdn.bixi.com/wp-content/uploads/2023/06/Historique-BIXI-2015.zip
start download and extract https://s3.ca-cen

KeyboardInterrupt: 

## Creating indexes

For [UC006](https://github.com/Monitoring-Mtl/Frontend/issues/114), aggregations will be frequently performed on the `DURATIONMS` columns. For that reason, we create an index on `DURATIONSMS` for faster lookups. Depending on the use case, more columns can be indexed.

In [ ]:
db[config.BIXI_TRIP_COLLECTION].create_index("DURATIONMS")
db[config.BIXI_TRIP_COLLECTION].create_index("STARTTIMEMS")
db[config.BIXI_TRIP_COLLECTION].create_index("ENDTIMEMS")

'ENDTIMEMS_1'

## DB statistics

In [ ]:
import datetime


def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"

for collection_name in db.list_collection_names():
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"Collection: {collection_name} as of {current_time}")
    stats = db.command("collStats", collection_name)
    print(f"Collection: {collection_name}")
    print(f"Storage size: {sizeof_fmt(stats['storageSize'])}")
    print(f"Documents: {stats['count']}")
    print(f"Avg. document size: {sizeof_fmt(stats['avgObjSize'])}")
    print(f"Indexes: {stats['nindexes']}")
    print(f"Total index size: {sizeof_fmt(stats['totalIndexSize'])}\n")

Collection: historic_data_urls as of 2024-04-04 20:00:10
Collection: historic_data_urls
Storage size: 36.0KiB
Documents: 3
Avg. document size: 128.0B
Indexes: 1
Total index size: 36.0KiB

Collection: station_locations as of 2024-04-04 20:00:10
Collection: station_locations
Storage size: 116.0KiB
Documents: 1103
Avg. document size: 119.0B
Indexes: 1
Total index size: 92.0KiB

Collection: trips as of 2024-04-04 20:00:10
Collection: trips
Storage size: 1.3GiB
Documents: 20883589
Avg. document size: 174.0B
Indexes: 4
Total index size: 485.3MiB

